In [6]:
import random
from bokeh import events
from bokeh.io import output_notebook
from bokeh.layouts import Row, Column
from bokeh.models import Button, CustomJS, PreText
from bokeh.plotting import figure, show, ColumnDataSource

output_notebook()
TOOLS = "pan,wheel_zoom,box_zoom,reset,save"
COLORS = ['red', 'green', 'blue']
POINTS_COUNT = 50
POINTS_SIZE = 10

Loading BokehJS ...

In [7]:
def saveCallback(colors, source, state):
    return CustomJS(args=dict(colors=colors, source=source, state=state), code="""
    const data = source.data;
    state.text = 'saving...';
    const savedData = {};
    colors.forEach(color => { savedData[color] = []; });
    data['color'].forEach((d, i) => { savedData[d].push([data['X'][i], data['Y'][i]]); });

    const a = document.createElement("a");
    document.body.appendChild(a);
    a.download = 'color_test.json';
    a.href = "data:text/plain," + encodeURIComponent(JSON.stringify(savedData));
    a.click();
    a.remove();
    const now = new Date();
    state.text = 'saved / ' + now.getFullYear() + '.' + (now.getMonth()+1) + '.' + now.getDate() + ' ' + now.getHours() + ':' + now.getMinutes() + ':' + now.getSeconds();
""")

def buttonPlot(source):
    saveState = PreText(text='unsaved', width=400 , height=20, margin = (30, 0, 0, 10))
    saveButton = Button(label="Save", button_type="primary", width=100, height=30, margin = (20, 0, 0, 50))
    saveButton.js_on_event(events.ButtonClick, saveCallback(COLORS, source, saveState))
    return Row(saveButton,  saveState)

In [8]:
def doubleTapCallback(colors, source, size):
    return CustomJS(args=dict(colors=colors, source=source, size=size), code="""
    const data = source.data;
    const x = cb_obj['x'];
    const y = cb_obj['y'];
    for(let i = 0; i < data['color'].length; i++){
      if (x-size <= data['X'][i] && data['X'][i] <= x+size && y-size <= data['Y'][i] && data['Y'][i] <= y+size) {
        let oldColorIdx = colors.indexOf(data['color'][i]);
        let newColor = colors[(oldColorIdx+1)%(colors.length)];
        data['color'][i] = newColor;
        break;
      }
    }
    source.change.emit();
""")

def scatterPlot(source):
    plot = figure(tools=TOOLS, plot_width=500, plot_height=300)
    plot.scatter(x='X', y='Y', color='color', source=source, size=POINTS_SIZE)
    plot.js_on_event('doubletap', doubleTapCallback(COLORS, source, POINTS_SIZE/10))
    return plot

In [9]:
X = [random.randint(0, POINTS_COUNT) for _ in range(POINTS_COUNT)]
Y = [random.randint(0, POINTS_COUNT) for _ in range(POINTS_COUNT)]
color = [random.choice(COLORS) for _ in range(POINTS_COUNT)]
source = ColumnDataSource(dict(X=X, Y=Y, color=color))

In [10]:
button = buttonPlot(source)
plot = scatterPlot(source)
result = Column(button, plot)
show(result)